In [26]:
# study_controller.ipynb
# 一个用于系统性对比研究的实验控制台 (已升级，支持随机混态)

# In[1]:
# =============================================================================
# --- 1. 导入和设置 (Imports and Setup) ---
# =============================================================================
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import multiprocessing
from functools import partial
import time

# 从我们创建的模块中导入函数
from cspsa import run_cspsa_chsh_simulation
from sga import run_sga_chsh_simulation

# 将算法函数放入一个字典中，方便调用
ALGORITHMS = {
    'CSPSA': run_cspsa_chsh_simulation,
    'SGA': run_sga_chsh_simulation,
}

In [27]:

# In[2]:
# =============================================================================
# --- 2. 状态生成函数 (State Generation Functions) ---
# =============================================================================

def generate_werner_state(theta_deg, p):
    """根据给定的theta和纯度p，生成一个Werner态。"""
    ket0, ket1 = qt.basis(2, 0), qt.basis(2, 1)
    theta_rad = np.deg2rad(theta_deg)

    psi_pure = np.cos(theta_rad) * qt.tensor(ket0, ket0) + np.sin(theta_rad) * qt.tensor(ket1, ket1)
    rho_pure = qt.ket2dm(psi_pure)

    # 混合白噪声
    rho_mixed = p * rho_pure + (1 - p) * qt.tensor(qt.qeye(2), qt.qeye(2)) / 4.0
    return rho_mixed

def generate_random_pure_state():
    """根据Haar测度，生成一个随机的2-qubit纯态。"""
    # N=4表示2-qubit系统 (2*2)
    # 修正：使用新版QuTiP API
    ket = qt.rand_ket(N=4, dims=[[2, 2], [1, 1]], distribution='haar')
    return qt.ket2dm(ket)

def generate_random_mixed_state():
    """根据Hilbert-Schmidt测度，生成一个随机的2-qubit混态。"""
    # 修正：适配旧版QuTiP API，使用位置参数N，并手动设置dims
    dm = qt.rand_dm(4, distribution='hs')
    dm.dims = [[2, 2], [2, 2]] # 确保正确的张量积结构
    return dm


# In[3]:
# =============================================================================
# --- 3. ✅ 主配置区域 (Master Configuration Area) ---
# 这是您进行控制变量法实验的“控制面板”
# =============================================================================

# --- 变量1: 选择要运行的算法 ---
ALGORITHMS_TO_RUN = [
    'CSPSA',
    'SGA',
]

# --- 变量2: 定义要测试的量子态 ---
# 您可以注释掉不想运行的行
STATES_TO_RUN = {
    # 您文章中定义的6个态
    "Pure, θ=45°":   generate_werner_state(45.0, 1.0),
   # "Pure, θ=30°":   generate_werner_state(30.0, 1.0),
    #"Mixed, p=0.5, θ=30°": generate_werner_state(30.0, 0.5),
    #"Mixed, p=0.5, θ=22.5°": generate_werner_state(22.5, 0.5),

    # 新增的随机态，用于研究一般情况
    #"Random Pure State 1": generate_random_pure_state(),
    #"Random Mixed State 1 (HS)": generate_random_mixed_state()
}

# --- 变量3: 定义要对比的超参数组 ---
HPARAMS_SETS = {
    'default': {
        'cspsa': (1.0, 1, 0.25, 1/6.0), # (a, s, b, r)
        'sga':   (1.0, 1, 0.25, 1/6.0),
    },
    'a+': {
        'cspsa': (1.5, 1.25, 0.25, 1/3.0), # (a, s, b, r)
        'sga':   (2.5, 1.0,  0.5,  1/6.0),
    },
     's+': {
        'cspsa': (1.0, 1.5, 0.25, 1/3.0), # (a, s, b, r)
        'sga':   (2.0, 1.25,  0.5,  1/6.0),
    },
    'b+': {
        'cspsa': (1.0, 1.25, 0.5, 1/3.0), # (a, s, b, r)
        'sga':   (2.0, 1.0,  0.75,  1/6.0),
    },
    'r+': {
        'cspsa': (1.0, 1.25, 0.25, 2/3.0), # (a, s, b, r)
        'sga':   (2.0, 1.0,  0.5,  1/3.0),
    },
    #'s+a-b+r-': {
   #     'cspsa': (0.75, 1.5, 0.5, 1/6.0), # (a, s, b, r)
    #    'sga':   (2.0, 1.0,  0.5,  1/6.0),
    #},
   'a-': {
        'cspsa': (0.75, 1.25, 0.25, 1/3.0), # (a, s, b, r)
        'sga':   (1.5, 1.0,  0.5,  1/6.0),
    },
     's-': {
        'cspsa': (1.0, 1.0, 0.25, 1/3.0), # (a, s, b, r)
        'sga':   (2.0, 0.75,  0.5,  1/6.0),
    },
    'b-': {
        'cspsa': (1.0, 1.25, 0.2, 1/3.0), # (a, s, b, r)
        'sga':   (2.0, 1.0,  0.25,  1/6.0),
    },
    'r-': {
        'cspsa': (1.0, 1.25, 0.25, 1/6.0), # (a, s, b, r)
        'sga':   (2.0, 1.0,  0.5,  1/12.0),
    },
}

# --- 其他全局模拟参数 ---
COMMON_CONFIG = {
    'iterations': 200,
    'num_repetitions':30,
}

In [28]:

# In[4]:
# =============================================================================
# --- 4. 实验执行循环 (Experiment Execution Loop) ---
# =============================================================================
if __name__ == '__main__':
    all_results = []
    start_time = time.time()

    # 外层循环遍历不同的超参数组
    for hparam_name, hparam_values in HPARAMS_SETS.items():
        print(f"\n{'='*20} RUNNING WITH HPARAMS: '{hparam_name}' {'='*20}")

        # 中层循环遍历不同的量子态
        for state_name, state_rho in STATES_TO_RUN.items():

            # 内层循环遍历不同的算法
            for algo_name in ALGORITHMS_TO_RUN:
                print(f"\n----- Running: [Algo: {algo_name}] | [State: {state_name}] -----")

                # a. 准备本次运行的配置
                current_config = {
                    'state': state_rho,
                    'hparams': hparam_values,
                    **COMMON_CONFIG
                }

                # b. 并行运行模拟
                num_cores = multiprocessing.cpu_count()
                worker_function = partial(ALGORITHMS[algo_name], current_config)

                with multiprocessing.Pool(processes=num_cores) as pool:
                    histories = pool.map(worker_function, range(COMMON_CONFIG['num_repetitions']))

                # c. 计算平均结果并存储
                mean_history = np.mean(np.abs(histories), axis=0)

                all_results.append({
                    'hparam_set': hparam_name,
                    'state_name': state_name,
                    'algo_name': algo_name,
                    'mean_history': mean_history
                })

    end_time = time.time()
    print(f"\n\n{'='*20} ALL SIMULATIONS COMPLETE IN {end_time - start_time:.2f} SECONDS {'='*20}")


==================== RUNNING WITH HPARAMS: 'default' ====================

----- Running: [Algo: CSPSA] | [State: Pure, θ=45°] -----
  - [CSPSA] Starting Trial 11...
  - [CSPSA] Starting Trial 21...

----- Running: [Algo: SGA] | [State: Pure, θ=45°] -----
  - [SGA] Starting Trial 11...
  - [SGA] Starting Trial 21...

==================== RUNNING WITH HPARAMS: 'a+' ====================

----- Running: [Algo: CSPSA] | [State: Pure, θ=45°] -----
  - [CSPSA] Starting Trial 11...
  - [CSPSA] Starting Trial 21...

----- Running: [Algo: SGA] | [State: Pure, θ=45°] -----
  - [SGA] Starting Trial 11...
  - [SGA] Starting Trial 21...

==================== RUNNING WITH HPARAMS: 's+' ====================

----- Running: [Algo: CSPSA] | [State: Pure, θ=45°] -----
  - [CSPSA] Starting Trial 11...
  - [CSPSA] Starting Trial 21...

----- Running: [Algo: SGA] | [State: Pure, θ=45°] -----
  - [SGA] Starting Trial 11...
  - [SGA] Starting Trial 21...

==================== RUNNING WITH HPARAMS: 'b+' ====

In [29]:
# In[5]:
# =============================================================================
# --- 5. 结果绘图 (Results Plotting) - 最终出版级版本 ---
# =============================================================================

# --- a. 定义视觉映射 ---
# 1. 为每个超参数组定义一种鲜明的颜色
hparam_names = list(HPARAMS_SETS.keys())
colors = plt.cm.rainbow(np.linspace(0, 1, len(hparam_names)))
HPARAM_COLORS = {name: colors[i] for i, name in enumerate(hparam_names)}

# 2. 为每个量子态定义一种线型
state_names = list(STATES_TO_RUN.keys())
linestyles = ['-', '--', ':']
STATE_LINESTYLES = {name: linestyles[i % len(linestyles)] for i, name in enumerate(state_names)}

# --- b. 创建子图 ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=True, dpi=600)
iterations_axis = np.arange(1, COMMON_CONFIG['iterations'] + 2)

# --- c. 循环绘制数据到对应的子图 ---
for result in all_results:
    target_ax = ax1 if result['algo_name'] == 'CSPSA' else ax2
    color = HPARAM_COLORS[result['hparam_set']]
    linestyle = STATE_LINESTYLES[result['state_name']]

    # 我们只在图例中显示标记，图中只画线
    target_ax.plot(iterations_axis, result['mean_history'],
                   color=color,
                   linestyle=linestyle,
                   linewidth=3) # 增加线宽

# --- d. 设置子图的标题和标签 ---
ax1.text(0.025, 0.95, '(a) CSPSA', transform=ax1.transAxes, fontsize=24, fontweight='bold', va='top')
ax2.text(0.025, 0.95, '(b) SGA', transform=ax2.transAxes, fontsize=24, fontweight='bold', va='top')

for ax in [ax1, ax2]:
    ax.set_xlabel("iteration", fontsize=30)
    ax.grid(False)
    ax.set_ylim(bottom=0)
    # 1. 增大刻度字体
    ax.tick_params(axis='both', which='major', labelsize=30)

ax1.set_ylabel("Violation", fontsize=30)

# --- e. 创建独立的、清晰的图例 ---
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# 图例1: 颜色代表超参数
legend_elements_hparams = [Patch(facecolor=HPARAM_COLORS[name], label=name)
                           for name in hparam_names]

# 图例2: 线型代表量子态
legend_elements_states = [Line2D([0], [0], color='black', linestyle=STATE_LINESTYLES[name], lw=2.5, label=name)
                          for name in state_names]

# 图例3: 算法 (为了清晰，可以省略，因为已在子图标题中说明)
# legend_elements_algos = [ ... ]

# 将图例放在图表下方，并确保不重叠
# 2. 调整图例位置防止重叠
fig.legend(handles=legend_elements_hparams,
           loc='upper center',
           bbox_to_anchor=(0.32, 0.36), # 放置在底部中央
           ncol=5,                   # 分为多列
           fontsize=18,
           title='Hyperparameter Sets', title_fontsize=20, frameon=False)

fig.legend(handles=legend_elements_states,
           loc='upper center',
           bbox_to_anchor=(0.77, 0.41), # 在第一个图例下方
           ncol=1,
           fontsize=18,
           title='Quantum States', title_fontsize=20, frameon=False)


# --- f. 调整整体布局并显示 ---
# 调整rect和使用subplots_adjust来为图例留出足够的空间
fig.tight_layout(rect=[0, 0.1, 1, 0.93])

# 3. 保存为高质量矢量图PDF
plt.savefig("fig2.pdf", format='pdf', bbox_inches='tight')

plt.show()